In [ ]:
import firebase_admin #type:ignore
from firebase_admin import credentials, firestore #type:ignore
import pandas as pd #type:ignore
import numpy as np #type:ignore
import tensorflow as tf #type:ignore
from tensorflow.keras.models import Sequential #type:ignore
from tensorflow.keras.layers import LSTM, Dense, Dropout #type:ignore
from sklearn.preprocessing import MinMaxScaler #type:ignore
import matplotlib.pyplot as plt #type:ignore

# Step 1: Initialize Firebase Admin SDK (This needs to be done only once)
try:
    cred = credentials.Certificate("path/to/your-firebase-adminsdk.json")  # Replace with your Firebase credentials file
    firebase_admin.initialize_app(cred)
except:
    print("Firebase already initialized or failed to initialize")

# Step 2: Fetch Data from Firebase Firestore
db = firestore.client()
data_ref = db.collection('your_collection_name')  # Replace with your Firebase collection name

# Fetch data
docs = data_ref.stream()

# Store the timestamp and energy values in a list
data = []
for doc in docs:
    timestamp = doc.get('timestamp')  # Ensure your Firestore document contains 'timestamp' field
    energy = doc.get('energy')        # Ensure your Firestore document contains 'energy' field
    data.append({'timestamp': timestamp, 'energy': energy})

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Step 3: Prepare the data for LSTM
energy_data = df[['energy']].values

# Feature scaling (Normalization)
scaler = MinMaxScaler(feature_range=(0, 1))
energy_scaled = scaler.fit_transform(energy_data)

# Create sequences for LSTM (look-back window)
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(seq_length, len(data)):
        sequences.append(data[i-seq_length:i, 0])
        labels.append(data[i, 0])
    return np.array(sequences), np.array(labels)

SEQ_LENGTH = 60  # You can adjust this for how many past data points to consider
X, y = create_sequences(energy_scaled, SEQ_LENGTH)

# Reshape for LSTM input (samples, timesteps, features)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Step 4: Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 5: Train the model
history = model.fit(X, y, epochs=20, batch_size=32)

# Step 6: Plot the training loss (optional)
plt.plot(history.history['loss'], label='Training Loss')
plt.title('Model Loss During Training')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Step 7: Predictions (optional)
predictions = model.predict(X)
predictions = scaler.inverse_transform(predictions)

# Show prediction for next step
print("Predicted energy for next timestep:", predictions[-1])

# Optional: Plot actual vs predicted (to visualize performance)
plt.plot(df.index[SEQ_LENGTH:], energy_data[SEQ_LENGTH:], label='True Energy Data', color='blue')
plt.plot(df.index[SEQ_LENGTH:], predictions, label='Predicted Energy Data', color='red')
plt.title('Actual vs Predicted Energy')
plt.xlabel('Time')
plt.ylabel('Energy')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential #type:ignore
from tensorflow.keras.layers import LSTM, Dense #type:ignore
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import json

# Load the dataset
data = pd.read_csv('Datasets/data.csv')

# Convert 'timestamp' to datetime format
data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Set the 'timestamp' column as the index for time-series modeling
data.set_index('timestamp', inplace=True)

# Ensure 'unitConsumption' column is numeric; convert if necessary
data['unitConsumption'] = pd.to_numeric(data['unitConsumption'], errors='coerce')

# Handle any missing values created during conversion
data.fillna(method='ffill', inplace=True)  # Forward fill for missing values

# Normalize the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data['unitConsumption'] = scaler.fit_transform(data['unitConsumption'].values.reshape(-1, 1))

# Prepare the dataset for LSTM
def create_dataset(data, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step)]  # Corrected: Now accessing the data properly
        X.append(a)
        Y.append(data[i + time_step])
    return np.array(X), np.array(Y)

# Specify the time step (window size)
time_step = 10  # Can be adjusted depending on how much past data you want to use for prediction
X, y = create_dataset(data['unitConsumption'].values, time_step)

# Reshape the data for LSTM [samples, timesteps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], 1), activation='relu'))
model.add(Dense(1))  # Output layer for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions and the actual test data back to the original scale
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred = scaler.inverse_transform(y_pred)

# Save predictions and actual values to a JSON file
output_data = {
    "actual": y_test.flatten().tolist(),
    "predicted": y_pred.flatten().tolist()
}

# Specify the output file path
output_file = 'output.json'

# Write to JSON file
with open(output_file, 'w') as f:
    json.dump(output_data, f, indent=4)

# Print success message
print(f"Predictions and actual values saved to {output_file}")

# Optionally, plot the actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(y_pred, label='Predicted unitConsumption', marker='o')
plt.xlabel('Time')
plt.ylabel('unitConsumption')
plt.title('unitConsumption: Actual vs. Predicted')
plt.legend()
plt.show()


## Automatic trigger on specific time code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import json
from apscheduler.schedulers.blocking import BlockingScheduler
import datetime

# Define the ML algorithm function
def run_ml_algorithm():
    print("Running ML algorithm at", datetime.datetime.now())

    # Load the dataset
    data = pd.read_csv('Datasets/data.csv')

    # Convert 'timestamp' to datetime format
    data['timestamp'] = pd.to_datetime(data['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    # Set the 'timestamp' column as the index for time-series modeling
    data.set_index('timestamp', inplace=True)

    # Ensure 'unitConsumption' column is numeric; convert if necessary
    data['unitConsumption'] = pd.to_numeric(data['unitConsumption'], errors='coerce')

    # Handle any missing values created during conversion
    data.fillna(method='ffill', inplace=True)  # Forward fill for missing values

    # Normalize the data using MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    data['unitConsumption'] = scaler.fit_transform(data['unitConsumption'].values.reshape(-1, 1))

    # Prepare the dataset for LSTM
    def create_dataset(data, time_step=1):
        X, Y = [], []
        for i in range(len(data) - time_step - 1):
            a = data[i:(i + time_step)]  # Corrected: Now accessing the data properly
            X.append(a)
            Y.append(data[i + time_step])
        return np.array(X), np.array(Y)

    # Specify the time step (window size)
    time_step = 10  # Can be adjusted depending on how much past data you want to use for prediction
    X, y = create_dataset(data['unitConsumption'].values, time_step)

    # Reshape the data for LSTM [samples, timesteps, features]
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Build the LSTM model
    model = Sequential()
    model.add(LSTM(50, input_shape=(X_train.shape[1], 1), activation='relu'))
    model.add(Dense(1))  # Output layer for regression

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

    # Make predictions
    y_pred = model.predict(X_test)

    # Inverse transform the predictions and the actual test data back to the original scale
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
    y_pred = scaler.inverse_transform(y_pred)

    # Save predictions and actual values to a JSON file
    output_data = {
        "actual": y_test.flatten().tolist(),
        "predicted": y_pred.flatten().tolist()
    }

    # Specify the output file path
    output_file = 'output.json'

    # Write to JSON file
    with open(output_file, 'w') as f:
        json.dump(output_data, f, indent=4)

    # Print success message
    print(f"Predictions and actual values saved to {output_file}")

    # Optionally, plot the actual vs predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(y_pred, label='Predicted unitConsumption', marker='o')
    plt.xlabel('Time')
    plt.ylabel('unitConsumption')
    plt.title('unitConsumption: Actual vs. Predicted')
    plt.legend()
    plt.show()
    
    scheduler.shutdown(wait=False)
# Create an instance of the scheduler
scheduler = BlockingScheduler()

# Schedule the ML algorithm at 10 PM daily
scheduler.add_job(run_ml_algorithm, 'cron', hour=21, minute=52)

# Start the scheduler
try:
    scheduler.start()
except (KeyboardInterrupt, SystemExit):
    pass
